<a href="https://colab.research.google.com/github/icecat14159/PL-Repo./blob/main/HW06_%E6%9B%B8%E7%B1%8D%E6%B8%85%E5%96%AE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread gradio pandas matplotlib
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import datetime
import io
import matplotlib.pyplot as plt
import gradio as gr

# 授權連線 Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 連結你的試算表
sheet_url = "https://docs.google.com/spreadsheets/d/1fGjPVqPHt3flo-LxBNU9EZl_ZMvg-I8UXfMhIY-jDNA/edit?usp=sharing"
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1

In [2]:
# 定義欄位名稱
COLUMNS = ["個人評級", "書籍名稱", "閱讀進度", "最後紀錄日期", "書籍類型", "相關作品"]

# 讀取 Google Sheet
def read_data():
    records = worksheet.get_all_records()
    df = pd.DataFrame(records)
    if df.empty:
        df = pd.DataFrame(columns=COLUMNS)
    return df

# 寫回 Google Sheet
def write_data(df):
    worksheet.clear()
    worksheet.append_row(COLUMNS)
    worksheet.append_rows(df.values.tolist())

In [3]:
# 新增書籍紀錄
def add_record(rating, name, progress, date, genre, related):
    df = read_data()
    if not date:
        date = datetime.date.today().strftime("%Y/%m/%d")
    new_entry = {"個人評級": rating, "書籍名稱": name, "閱讀進度": progress,
                 "最後紀錄日期": date, "書籍類型": genre, "相關作品": related}
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    write_data(df)
    return df

# 修改紀錄
def edit_record(index, rating, name, progress, date, genre, related):
    df = read_data()
    if index < 0 or index >= len(df):
        return "無效索引"
    df.loc[index] = [rating, name, progress, date, genre, related]
    write_data(df)
    return df

# 刪除紀錄
def delete_record(index):
    df = read_data()
    if index < 0 or index >= len(df):
        return "無效索引"
    df = df.drop(index).reset_index(drop=True)
    write_data(df)
    return df

In [4]:
import base64
# 匯出 CSV
def export_csv():
    df = read_data()
    # 使用 utf-8-sig 可讓 Excel 正確讀中文
    csv_data = df.to_csv(index=False, encoding="utf-8-sig")
    b64 = base64.b64encode(csv_data.encode("utf-8-sig")).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="books.csv">下載 CSV 檔案</a>'
    return href

# 匯出 JSON
def export_json():
    df = read_data()
    json_data = df.to_json(orient="records", force_ascii=False)
    b64 = base64.b64encode(json_data.encode("utf-8")).decode()
    href = f'<a href="data:application/json;base64,{b64}" download="books.json">下載 JSON 檔案</a>'
    return href

# 匯入 CSV
def import_csv(file):
    df = pd.read_csv(file.name)
    write_data(df)
    return df

# 匯入 JSON
def import_json(file):
    df = pd.read_json(file.name)
    write_data(df)
    return df

In [5]:
export_csv()

'<a href="data:file/csv;base64,77u/5YCL5Lq66KmV57SaLOabuOexjeWQjeeosSzplrHoroDpgLLluqYs5pyA5b6M57SA6YyE5pel5pyfLOabuOexjemhnuWeiyznm7jpl5zkvZzlk4EKUyzpgYrmiLLkurrnlJ8sMTIsMjAyNC8xMi8xMCzlsI/oqqosCkEs5LiN5q2j57aT55qE6a2U6KGT6Kyb5birLDIxLDIwMjUvMTAvOSzlsI/oqqosCkMs6Ieq56ix6LOi6ICF5byf5a2Q55qE6LOi6ICFLDE2LDIwMjUvMS8xLOWwj+iqqiwKUyzni7zoiIfovpvpppnmlpksMjAsMjAyNS8xMC8yNSzlsI/oqqosClMs6JGs6YCB6ICF56aP5Yip6IGvLDE0MiwyMDI1LzEwLzIwLOa8q+eVqywKUyznuL3kuYvlsLHmmK/pnZ7luLjlj6/mhJssMzE3LDIwMjUvNS84LOa8q+eVqywKQizku4Hnp5HnmoTnt6josqzlhpLpmqroqJgsMjUsMjAyNS81LzUs5ryr55WrLApBLOaVl+WMl+Wls+inkuWkquWkmuS6hiw4LDIwMjUtMTAtMjYs5bCP6KqqLAo=" download="books.csv">下載 CSV 檔案</a>'

In [6]:
def advanced_query(rating=None, start_date=None, end_date=None, genre=None):
    df = read_data()
    if rating:
        df = df[df["個人評級"] == rating]
    if start_date:
        df = df[df["最後紀錄日期"] >= start_date]
    if end_date:
        df = df[df["最後紀錄日期"] <= end_date]
    if genre:
        df = df[df["書籍類型"].str.contains(genre, na=False)]
    return df

In [7]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('models/gemini-flash-latest')

In [8]:
def gemini_analysis(analysis_request):
    df = read_data()
    # Convert DataFrame to CSV string
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
    csv_data = csv_buffer.getvalue()
    print("轉檔完成")
    # Construct the prompt for the Gemini model
    prompt = f"""以下是我的書籍資料，格式為 CSV：
{csv_data}

請根據這些資料，執行以下分析，字數控制在500字以內：
{analysis_request}
"""

    # Call the Gemini model
    print("傳送完成")
    response = gemini_model.generate_content(prompt)
    print("收到回覆")
    # Return the text response
    return response.text

In [9]:
response = gemini_model.generate_content("你好")
print(response.text)

你好！有什么我可以帮忙的吗？


In [10]:
gemini_analysis("分析我的閱讀頻率")

轉檔完成
傳送完成
收到回覆


'根據您提供的八筆書籍資料，儘管無法計算實際的每日閱讀量（因為缺乏起始日期），但可以透過「最後紀錄日期」分析您的閱讀頻率、活動強度及優先順序。\n\n### 1. 爆發性活動頻率極高 (當前活躍度)\n\n您的閱讀習慣呈現出強烈的「爆發性追蹤」特點。在您紀錄的八本書中，有四本的「最後紀錄日期」集中在 **2025年10月9日到10月26日** 之間。\n\n*   **最近更新集中：**《敗北女角太多了》(10/26)、《狼與辛香料》(10/25)、《葬送者福利聯》(10/20)、《不正經的魔術講師》(10/9)。\n\n這顯示您在極短的時間跨度內，同時對至少四部作品進行了進度更新，這說明您目前處於高度活躍的閱讀期，並且習慣於**多線程同步閱讀**，而非專注於單一書籍。\n\n### 2. 多元化與平衡的閱讀組合\n\n您的閱讀頻率並未偏向單一類型，而是保持了小說與漫畫的平衡追蹤：\n\n*   在最近的四次更新中，涵蓋了三部小說和一部漫畫。\n*   整體而言，您有五本小說和三本漫畫的紀錄，顯示您在不同媒體間切換自如，能維持對兩種形式的進度追蹤。\n\n### 3. 優先度與頻率的連結\n\n從紀錄日期可以看出，您對個人評價高的作品投入了最高的追蹤頻率：\n\n*   四本最近更新的書目中，有三本屬於 S 級或 A 級（《狼與辛香料》S、《葬送者福利聯》S、《不正經的魔術講師》A、《敗北女角太多了》A）。\n*   相比之下，評價較低的書籍（《自稱賢者弟子的賢者》C 級，紀錄日期為 2025/1/1；《仁科的編責冒險記》B 級，紀錄日期為 2025/5/5）的更新日期較舊。\n\n這強烈暗示您的閱讀頻率與**興趣/評級直接相關**：當您對某本書的興趣度高（S/A 級）時，會更頻繁地投入時間並紀錄進度。\n\n### 總結\n\n您的閱讀頻率屬於**高強度、多線程且優先級導向**的類型。您在特定時間段內會集中精力閱讀和追蹤多本不同媒介的書籍，並顯著優先處理您個人評價為 S 或 A 級的作品。當前（2025年10月）正是您閱讀活動最為頻繁的時期。'

In [11]:
def show_records():
    return read_data()

with gr.Blocks() as app:
    gr.Markdown("## 書籍管理清單系統")
    with gr.Tab("目前紀錄"):
        refresh_btn = gr.Button("重新整理")
        table = gr.Dataframe(value=read_data(), headers=COLUMNS, interactive=False)
        refresh_btn.click(fn=show_records, outputs=table)

    with gr.Tab("新增紀錄"):
        rating = gr.Dropdown(["S","A","B","C","D","E","F"], label="個人評級")
        name = gr.Textbox(label="書籍名稱")
        progress = gr.Textbox(label="閱讀進度")
        date = gr.Textbox(label="最後紀錄日期 (可留空自動填)")
        genre = gr.Textbox(label="書籍類型")
        related = gr.Textbox(label="相關作品")
        add_btn = gr.Button("新增紀錄")
        output_add = gr.Dataframe()
        add_btn.click(add_record, [rating,name,progress,date,genre,related], output_add)

    with gr.Tab("修改紀錄"):
        idx = gr.Number(label="紀錄索引(從0開始)")
        rating_e = gr.Dropdown(["S","A","B","C","D","E","F"], label="個人評級")
        name_e = gr.Textbox(label="書籍名稱")
        progress_e = gr.Textbox(label="閱讀進度")
        date_e = gr.Textbox(label="最後紀錄日期")
        genre_e = gr.Textbox(label="書籍類型")
        related_e = gr.Textbox(label="相關作品")
        edit_btn = gr.Button("修改紀錄")
        output_edit = gr.Dataframe()
        edit_btn.click(edit_record, [idx,rating_e,name_e,progress_e,date_e,genre_e,related_e], output_edit)

    with gr.Tab("刪除紀錄"):
        del_idx = gr.Number(label="紀錄索引(從0開始)")
        del_btn = gr.Button("刪除")
        output_del = gr.Dataframe()
        del_btn.click(delete_record, del_idx, output_del)

    with gr.Tab("匯入/匯出"):
        csv_export = gr.Button("匯出CSV")
        json_export = gr.Button("匯出JSON")
        csv_import = gr.File(label="匯入CSV")
        json_import = gr.File(label="匯入JSON")
        out_csv = gr.HTML()
        out_json = gr.HTML()
        out_import = gr.Dataframe()
        csv_export.click(export_csv, outputs=out_csv)
        json_export.click(export_json, outputs=out_json)
        csv_import.change(import_csv, inputs=csv_import, outputs=out_import)
        json_import.change(import_json, inputs=json_import, outputs=out_import)

    with gr.Tab("進階查詢"):
        q_rating = gr.Dropdown(["","S","A","B","C","D","E","F"], label="評級")
        q_start = gr.Textbox(label="起始日期 (YYYY/MM/DD)")
        q_end = gr.Textbox(label="結束日期 (YYYY/MM/DD)")
        q_genre = gr.Textbox(label="書籍類型關鍵字")
        query_btn = gr.Button("查詢")
        query_out = gr.Dataframe()
        query_btn.click(advanced_query, [q_rating,q_start,q_end,q_genre], query_out)

    with gr.Tab("Gemini 分析"):
        analysis_request_input = gr.Textbox(label="分析請求", placeholder="請輸入您想對書籍資料進行的分析，例如：'分析我的閱讀頻率並給予建議'")
        analyze_gemini_btn = gr.Button("執行分析")
        gemini_analysis_output = gr.Markdown(label="分析結果")
        analyze_gemini_btn.click(gemini_analysis, inputs=analysis_request_input, outputs=gemini_analysis_output)

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa2e275e4b61e8fca4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
